# PDC Project
## Convolutional encoding / Viterbi decoding over an AWGN + erasure channel
### How to use :
Replace '80char.txt' below with the file to send.

Sequentially run all the cells.

In [23]:
# Imports

%load_ext autoreload
%autoreload 2
from encoder_utils import Encoder
from decoder_utils import Decoder
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# Parameters

L = 10 # Number of previous bits used
N = 24 # Number of output bits per input bit

In [25]:
# filenames

original_file  = '80char.txt'                             # << Insert here the name of the file to send
encoded_file   = 'encoded.txt'
channel_output = 'channel_output.txt'
decoded_file   = 'decoded.txt'

In [26]:
# Encoding-Decoding generator matrix (used by both the encoder and the decoder)

def create_generator_matrix(random=True):
    """
    Creates the generator matrix.
    We make sure that for each generator_polynomial (A line of the generator matrix), the first and the last bits are set to 1
    This is because we at least want b_i and b_{i-L} to be used to use the whole range L. The remaining coefficients are chosen
    at random as no method exists to optimally pick them deterministically.
    """
    if random:
        generator_matrix = np.random.choice([0, 1], size=(N, L+1))
        generator_matrix[:,0] = 1
        generator_matrix[:,L] = 1
    else:
        # Matrix found to have 100% success rate over 20 trials with (L=10, N=24)
        generator_matrix = np.array([[1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1],
                                     [1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1],
                                     [1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1],
                                     [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
                                     [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1],
                                     [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
                                     [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1],
                                     [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
                                     [1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1],
                                     [1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1],
                                     [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
                                     [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1],
                                     [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
                                     [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1],
                                     [1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],
                                     [1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1],
                                     [1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1],
                                     [1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1],
                                     [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1],
                                     [1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1],
                                     [1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1],
                                     [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1],
                                     [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1],
                                     [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]])
    return generator_matrix
generator_matrix = create_generator_matrix(random=False)
print('Generator matrix :\n', generator_matrix)

Generator matrix :
 [[1 1 1 0 1 0 0 0 1 1 1]
 [1 0 0 0 0 1 0 1 1 0 1]
 [1 1 1 0 1 1 1 1 0 0 1]
 [1 0 0 1 1 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 1 1 1]
 [1 1 1 1 1 0 1 1 1 1 1]
 [1 0 1 1 0 1 0 1 0 1 1]
 [1 0 1 1 0 0 0 1 0 0 1]
 [1 0 0 0 1 1 1 0 1 1 1]
 [1 0 0 0 0 1 1 0 1 0 1]
 [1 0 1 1 0 0 0 1 0 0 1]
 [1 1 1 1 0 1 1 1 1 0 1]
 [1 1 0 1 0 0 1 0 0 0 1]
 [1 1 0 0 0 1 0 1 1 1 1]
 [1 0 1 0 0 0 1 1 1 1 1]
 [1 1 0 1 0 1 0 0 1 1 1]
 [1 0 1 0 0 0 0 1 1 1 1]
 [1 1 0 0 0 1 1 1 1 0 1]
 [1 1 1 1 0 0 1 1 1 0 1]
 [1 1 0 1 1 0 1 1 0 0 1]
 [1 1 1 1 1 0 1 1 1 0 1]
 [1 1 1 0 1 0 1 1 1 0 1]
 [1 1 0 0 1 0 0 1 0 1 1]
 [1 0 1 1 1 0 0 0 1 1 1]]


In [27]:
# Instantiate the encoder and decoder

encoder = Encoder(input_file=original_file,  output_file=encoded_file, generator_matrix=generator_matrix, verbose=False)
decoder = Decoder(input_file=channel_output, output_file=decoded_file, generator_matrix=generator_matrix, verbose=False)

In [28]:
# Encode

encoder.encode()

Original file : b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy'
Total number of bits used : 30720


In [29]:
# AWGN + erasure channel

!python client.py --input_file {encoded_file} --output_file {channel_output} --srv_hostname iscsrv72.epfl.ch --srv_port 80

In [30]:
# Decode

decoded = decoder.decode()

Compact_values_length : 15360
Trellis height : 1024
Trellis width  : 641
Decoding ...
bytearray(b'Hello world, planet, solar system all the way to A-Centuri and the entire galaxy')
